In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
import os
os.chdir("/home/peppe/01_Study/01_University/Semester/2/Intro_to_ML/Project/Code")

In [3]:
from train import Trainer

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [6]:
# get data

# note: autougment doesn't seem to work very well 

from models_methods.data.preprocessing import Autoaugment_preprocess
transform = Autoaugment_preprocess(channels=3, resize_dim=(36,36), crop_dim=(28,28)) 


root = "/home/peppe/01_Study/01_University/Semester/2/Intro_to_ML/Project/data" # change this to your data directory

trainset = torchvision.datasets.CIFAR10(root=root, train=True,
                                      download=True, transform=transform.transform)

trainset, valset = torch.utils.data.random_split(trainset, [45000, 5000])

testset = torchvision.datasets.CIFAR10(root=root, train=False,
                                       download=True, transform=transform.transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=64,
                                        shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

data_loaders = {
    "train_loader": trainloader,
    "val_loader": valloader,
    "test_loader": testloader
}          

Files already downloaded and verified
Files already downloaded and verified


In [7]:
# create simple model 

import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [8]:
model = SimpleCNN()
optimizer = torch.optim.SGD

In [9]:
simple_net = Trainer(data_loaders=data_loaders,
                     dataset_name="CIFAR10",
                     model=model,
                     optimizer=torch.optim.SGD,
                     loss_fn=nn.CrossEntropyLoss(label_smoothing=0.1),
                     device=device,
                     seed=42,
                     exp_path="/home/peppe/01_Study/01_University/Semester/2/Intro_to_ML/Project/Code/experiments",
                     exp_name="test_16",
                     use_SAM=True
                    )

In [ ]:
simple_net.main(epochs=8, 
                log_interval=0.1)

In [22]:
!kill 21449

In [13]:
# visualize with tensorboard
%reload_ext tensorboard
%tensorboard --logdir=/home/peppe/01_Study/01_University/Semester/2/Intro_to_ML/Project/Code/experiments # experiment path

Reusing TensorBoard on port 6006 (pid 41527), started 0:00:02 ago. (Use '!kill 41527' to kill it.)